<a href="https://colab.research.google.com/github/hellokayas/Some-Programming-Samples/blob/master/MNIST_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torchvision

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
EPOCH = 10 
BATCH_SIZE= 64

 We will use the MNIST dataset already introduced in the previous post, which can be downloaded from The MNIST database page using torchvision.dataset. PyTorch Datasets are objects that return a single datapoint on request. Then it is passed on to a Dataloader which handles batching of datapoints and parallelism.

In [6]:
xy_trainPT = torchvision.datasets.MNIST(root='./data', 
             train=True, download=True,transform=
             torchvision.transforms.Compose(
             [torchvision.transforms.ToTensor()]))
xy_trainPT_loader = torch.utils.data.DataLoader(xy_trainPT, batch_size=BATCH_SIZE)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
image, _ = xy_trainPT[0] 
print(image.size())
image_flatten = image.view(image.shape[0], -1)
print (image_flatten.size())
torch.Size([1, 28, 28]) 
torch.Size([1, 784])

torch.Size([1, 28, 28])
torch.Size([1, 784])


torch.Size([1, 784])

In [8]:
modelPT= torch.nn.Sequential(            
         torch.nn.Linear(784,10),
         torch.nn.Sigmoid(), 
         torch.nn.Linear(10,10), 
         torch.nn.LogSoftmax(dim=1) 
         )

Loss functions, around 20 different in PyTorch, reside in the nn package and are implemented as an nn.Module subclass. Some of the common standard loss functions that we will use in this series are:
nn.MSELoss: The mean square error between arguments, which is the standard loss for regression problems.
nn.NLLLoss: It computes the “maximum likelihood” criteria and generaly are used in multi-class classification problems (as our MNIST example).
nn.CrossEntropyLoss: computes the same as nn.NLLLoss, however it expects raw scores for each class and applies LogSoftmax internally (while nn.NLLLoss expects to have log probabilities as the input).

In [15]:
criterion = torch.nn.NLLLoss()
loss = criterion(logps, labels)
optimizer = torch.optim.SGD(modelPT.parameters(), lr=0.01)

NameError: ignored

In [14]:
for e in range(EPOCH):
      running_loss = 0
      for images, labels in xy_trainPT_loader:
          images = images.view(images.shape[0], -1)
          output = modelPT(images)
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
          running_loss += loss.item()
      

NameError: ignored

Line 1: Usually, the training loop iterates over our data over and over again. Remember that one iteration over a full set of examples is called an epoch. The EPOCHS variable indicates the number of iterations over the full set of examples.
Line 2: We already presented that data is usually too large to fit into CPU or GPU memory at once, so it is split into batches of equal size. Every batch includes data samples and target labels, and both of them have to be tensors.
Line 3: To facilitate the entry of data into our neural network we must make a transformation of the input (image) from 2 dimensions (2D) to a vector of 1 dimension (1D).
Line 4: We pass each batch of image tensors into the model which will return a tensor having predictions for that batch, the forward pass.
Line 5: Having got the predictions we pass them into the cross-entropy Loss function ( criterion ) along with their actual labels and calculate the Loss. Usually, Loss functions accept two arguments: output from the network (prediction) and desired output (ground-truth data, which is also called the label of the data sample).
Some PyTorch’s Loss functions take class labels as their targets (e.g. NLLloss), so if we use them (as in our case), we don’t need to convert targets into one-hot vectors as we presented in the previous post (for ease the explanation).
Line 6: We do a backward pass using the Loss value in order to compute gradient of the Loss with respect to model parameters. After loss.backward() call is finished, we have the gradients accumulated.
Line 7: Now it’s time for the optimizer to modify the model parameters with the method step() that takes all gradients from the parameters and applies them.
Line 8: The last, but not least, piece of the training loop is our responsibility to zero gradients of parameters. Calling zero_grad() on our network clear the gradients of all optimized variables.